In [1]:
!pip install opencv-contrib-python --upgrade

CONNECT DRIVE

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


TẠO FILE kaggle.json

In [3]:
import json

data = {
    "username": "camhuy",   # đúng với username trên Kaggle
    "key": "KGAT_fd3cdad8ef19be80b840a5e6bdfdc3b5"  # dán API token vào đây
}

with open("kaggle.json", "w") as f:
    json.dump(data, f)

print("Đã tạo file kaggle.json!")

Đã tạo file kaggle.json!


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

TẢI DATASET CELEB A

In [5]:
!kaggle datasets download -d jessicali9530/celeba-dataset -p /content/celeba/
!unzip -q /content/celeba/celeba-dataset.zip -d /content/celeba/

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
License(s): other
 99% 1.32G/1.33G [00:07<00:00, 171MB/s]
100% 1.33G/1.33G [00:11<00:00, 123MB/s]


TẢI THÊM FILE identity_CelebA.txt VÌ TRONG DATASET THIẾU

In [6]:
!kaggle datasets download -d kymo9890/identity-celeba -p /content/celeba/
!unzip -q /content/celeba/identity-celeba.zip -d /content/celeba/

Dataset URL: https://www.kaggle.com/datasets/kymo9890/identity-celeba
License(s): unknown
  0% 0.00/943k [00:00<?, ?B/s]
100% 943k/943k [00:00<00:00, 732MB/s]


Import + THAM SỐ CHUNG



In [7]:
import os, random, shutil, json
from collections import defaultdict
from tqdm import tqdm
import cv2

RAW_IMG_DIR = "/content/celeba/img_align_celeba/img_align_celeba"
IDENTITY_FILE = "/content/celeba/identity_CelebA.txt"

IMG_SIZE = 200
IMAGES_PER_ID = 5
MAX_IDS = 200


Load identity map

In [8]:
id_map = defaultdict(list)

with open(IDENTITY_FILE) as f:
    for line in f:
        img, pid = line.strip().split()
        id_map[int(pid)].append(img)

print("Total identities:", len(id_map))


Total identities: 10177


LOAD Haar Cascade

In [9]:
CASCADE_PATH = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(CASCADE_PATH)


Tham số tối ưu detect

In [10]:
DETECT_SIZE = 300
SCALE_FACTOR = 1.2
MIN_NEIGHBORS = 4


Hàm detect_face_fast

In [11]:
def detect_face_fast(img_bgr):
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    h, w = gray.shape
    scale = DETECT_SIZE / max(h, w)
    small = cv2.resize(gray, (int(w*scale), int(h*scale)))

    faces = face_cascade.detectMultiScale(
        small,
        scaleFactor=SCALE_FACTOR,
        minNeighbors=MIN_NEIGHBORS
    )

    if len(faces) == 0:
        return None

    x, y, fw, fh = faces[0]

    x = int(x / scale)
    y = int(y / scale)
    fw = int(fw / scale)
    fh = int(fh / scale)

    face = gray[y:y+fh, x:x+fw]
    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

    return face


Split ratio

In [12]:
SPLIT_RATIO = {
    "train": 0.7,
    "val": 0.15,
    "test": 0.15
}


Tạo cấu trúc output

In [13]:
OUTPUT_DIR = "/content/lbph_dataset"

SPLIT_DIRS = {
    "train": os.path.join(OUTPUT_DIR, "train"),
    "val": os.path.join(OUTPUT_DIR, "val"),
    "test": os.path.join(OUTPUT_DIR, "test")
}

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

for d in SPLIT_DIRS.values():
    os.makedirs(d, exist_ok=True)


VÒNG LẶP TẠO DATASET

In [15]:
selected_ids = list(id_map.keys())[:MAX_IDS]

label_map = {}
current_label = 0

MIN_TRAIN = 3
MIN_VAL   = 1
MIN_TEST  = 1

for pid in tqdm(selected_ids):
    imgs = id_map[pid]
    random.shuffle(imgs)
    imgs = imgs[:IMAGES_PER_ID]

    # --------- detect trước, cache face ----------
    detected_faces = []
    for img_name in imgs:
        img_path = os.path.join(RAW_IMG_DIR, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        face = detect_face_fast(img)
        if face is not None:
            detected_faces.append(face)

    # bỏ class nếu detect không đủ ảnh
    if len(detected_faces) < (MIN_TRAIN + MIN_VAL + MIN_TEST):
        continue

    label = current_label
    label_map[pid] = label
    current_label += 1

    # --------- split an toàn ----------
    n = len(detected_faces)

    n_train = max(MIN_TRAIN, int(n * SPLIT_RATIO["train"]))
    n_val   = max(MIN_VAL,   int(n * SPLIT_RATIO["val"]))
    n_test  = n - n_train - n_val

    if n_test < MIN_TEST:
        n_test = MIN_TEST
        n_train = n - n_val - n_test

    # đảm bảo tổng = n
    while n_train + n_val + n_test > n:
        n_train -= 1

    random.shuffle(detected_faces)

    split_faces = {
        "train": detected_faces[:n_train],
        "val":   detected_faces[n_train:n_train+n_val],
        "test":  detected_faces[n_train+n_val:]
    }

    # --------- ghi file (chỉ tạo folder khi có ảnh) ----------
    for split, faces in split_faces.items():
        class_dir = os.path.join(SPLIT_DIRS[split], str(label))
        os.makedirs(class_dir, exist_ok=True)

        for i, face in enumerate(faces):
            cv2.imwrite(os.path.join(class_dir, f"{i}.jpg"), face)


100%|██████████| 200/200 [00:36<00:00,  5.44it/s]


Kiểm tra dataset

In [16]:
def count_images(base_dir):
    return sum(len(files) for _, _, files in os.walk(base_dir))

for s in SPLIT_DIRS:
    print(s, count_images(SPLIT_DIRS[s]))


train 447
val 149
test 149


Lưu dataset lên Google Drive

In [17]:
DRIVE_OUT = "/content/drive/MyDrive/LBPH_DATASET"

if os.path.exists(DRIVE_OUT):
    shutil.rmtree(DRIVE_OUT)

shutil.copytree(OUTPUT_DIR, DRIVE_OUT)

with open(os.path.join(DRIVE_OUT, "label_map.json"), "w") as f:
    json.dump(label_map, f, indent=2)
